El siguiente código ordena las causas de morbilidad de los pacientes atendidos durante el mes en los servicios de consulta externa, emergencia y hospitalización. Posteriormente clasifica por género y por grupo etario.

# Importación de librerías, carga de archivos y creación de DataFrames

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Designación de variables
CE = 'CE.xlsx'
ER_CE = 'EMERGENCIA.xlsx'
#ER_HO = 'EMERGENCIA (HO).xlsx'
HO = 'HOSPITALIZACION.xlsx'
pestaña = 'BD'

In [ ]:
# Designación de dataframes
df_CE = pd.read_excel(CE, sheet_name= pestaña)
df_ER_CE = pd.read_excel(ER_CE, sheet_name= pestaña)
#df_ER_HO = pd.read_excel(ER_HO, sheet_name= pestaña)é
df_HO = pd.read_excel(HO, sheet_name= pestaña)

# Transformación de los DataFrames

In [ ]:
# Extracción de las columnas necesarias
CE = df_CE[['EDAD', 'CEDULA', 'SEXO', 'CIE10', 'DESCRIPCION']]
ER_CE = df_ER_CE[['EDAD', 'CEDULA', 'SEXO', 'CIE10', 'DESCRIPCION']]
#ER_HO = df_ER_HO[['EDAD', 'CEDULA', 'SEXO', 'CIE10', 'DESCRIPCION']]
HO = df_HO[['EDAD', 'CEDULA', 'SEXO', 'CIE10', 'DESCRIPCION']]

In [ ]:
# Eliminación de duplicados por coincidencia de número de cédula y código CIE-10
CE = CE.drop_duplicates(subset=['CEDULA', 'CIE10'])
ER_CE = ER_CE.drop_duplicates(subset=['CEDULA', 'CIE10'])
#ER_HO = ER_HO.drop_duplicates(subset=['CEDULA', 'CIE10'])
HO = HO.drop_duplicates(subset=['CEDULA', 'CIE10'])

In [ ]:
# Eliminación de los registros con código CIE-10 Z y R, el primero corresponde a procedimientos y el segundo a signos clínicos
CE = CE[~CE['CIE10'].str.startswith(' Z')]
CE = CE[~CE['CIE10'].str.startswith(' R')]
ER_CE = ER_CE[~ER_CE['CIE10'].str.startswith(' Z')]
#ER_HO = ER_HO[~ER_HO['CIE10'].str.startswith(' Z')]
HO = HO[~HO['CIE10'].str.startswith(' Z')]

# Creación de la tabla de morbilidad por rangos de edad y género

In [ ]:
# Se definen los rangos de edad
bins = [0, 1, 5, 10, 15, 20, 50, 65, np.inf]
names = ['<1año', '1-4', '5-9', '10-14', '15-19', '20-49', '50-64', '65+']

# Se crea una nueva columna que indica el rango de edad
CE['RangoEdad'] = pd.cut(CE['EDAD'], bins, labels=names)
ER_CE['RangoEdad'] = pd.cut(ER_CE['EDAD'], bins, labels=names)
#ER_HO['RangoEdad'] = pd.cut(ER_HO['EDAD'], bins, labels=names)
HO['RangoEdad'] = pd.cut(HO['EDAD'], bins, labels=names)

# Se agrupan los datos por código CIE10, rango de edad y género
df_agrupado_CE = CE.groupby(['CIE10', 'RangoEdad', 'SEXO']).size().reset_index(name='conteo')
df_agrupado_ER_CE = ER_CE.groupby(['CIE10', 'RangoEdad', 'SEXO']).size().reset_index(name='conteo')
#df_agrupado_ER_HO = ER_HO.groupby(['CIE10', 'RangoEdad', 'SEXO']).size().reset_index(name='conteo')
df_agrupado_HO = HO.groupby(['CIE10', 'RangoEdad', 'SEXO']).size().reset_index(name='conteo')

# Se crean tablas dinámicas por cada servicio
df_pivot_CE = df_agrupado_CE.pivot_table(index=['CIE10'], columns=['RangoEdad', 'SEXO'], values='conteo', fill_value=0)
df_pivot_ER_CE = df_agrupado_ER_CE.pivot_table(index=['CIE10'], columns=['RangoEdad', 'SEXO'], values='conteo', fill_value=0)
#df_pivot_ER_HO = df_agrupado_ER_HO.pivot_table(index=['CIE10'], columns=['RangoEdad', 'SEXO'], values='conteo', fill_value=0)
df_pivot_HO = df_agrupado_HO.pivot_table(index=['CIE10'], columns=['RangoEdad', 'SEXO'], values='conteo', fill_value=0)

for name in names:
    for sex in ['M', 'F']:
        if (name, sex) not in df_pivot_CE.columns:
            df_pivot_CE[(name, sex)] = 0

for name in names:
    for sex in ['M', 'F']:
        if (name, sex) not in df_pivot_ER_CE.columns:
            df_pivot_ER_CE[(name, sex)] = 0
"""
for name in names:
    for sex in ['M', 'F']:
        if (name, sex) not in df_pivot_ER_HO.columns:
            df_pivot_ER_HO[(name, sex)] = 0
"""
for name in names:
    for sex in ['M', 'F']:
        if (name, sex) not in df_pivot_HO.columns:
            df_pivot_HO[(name, sex)] = 0

# Se ordenan las columnas
df_pivot_CE = df_pivot_CE.sort_index(axis=1)
df_pivot_ER_CE = df_pivot_ER_CE.sort_index(axis=1)
#df_pivot_ER_HO = df_pivot_ER_HO.sort_index(axis=1)
df_pivot_HO = df_pivot_HO.sort_index(axis=1)

# Se añaden las columnas de total por género y total general
df_pivot_CE[('Total', 'M')] = df_pivot_CE.xs('M', level=1, axis=1).sum(axis=1)
df_pivot_CE[('Total', 'F')] = df_pivot_CE.xs('F', level=1, axis=1).sum(axis=1)
df_pivot_CE[('Total', 'TOTAL')] = df_pivot_CE[('Total', 'M')] + df_pivot_CE[('Total', 'F')]

df_pivot_ER_CE[('Total', 'M')] = df_pivot_ER_CE.xs('M', level=1, axis=1).sum(axis=1)
df_pivot_ER_CE[('Total', 'F')] = df_pivot_ER_CE.xs('F', level=1, axis=1).sum(axis=1)
df_pivot_ER_CE[('Total', 'TOTAL')] = df_pivot_ER_CE[('Total', 'M')] + df_pivot_ER_CE[('Total', 'F')]
"""
df_pivot_ER_HO[('Total', 'M')] = df_pivot_ER_HO.xs('M', level=1, axis=1).sum(axis=1)
df_pivot_ER_HO[('Total', 'F')] = df_pivot_ER_HO.xs('F', level=1, axis=1).sum(axis=1)
df_pivot_ER_HO[('Total', 'TOTAL')] = df_pivot_ER_HO[('Total', 'M')] + df_pivot_ER_HO[('Total', 'F')]
"""
df_pivot_HO[('Total', 'M')] = df_pivot_HO.xs('M', level=1, axis=1).sum(axis=1)
df_pivot_HO[('Total', 'F')] = df_pivot_HO.xs('F', level=1, axis=1).sum(axis=1)
df_pivot_HO[('Total', 'TOTAL')] = df_pivot_HO[('Total', 'M')] + df_pivot_HO[('Total', 'F')]

In [ ]:
# Se ordena el DataFrame en orden descendente por 'TOTAL'
df_pivot_CE.sort_values(by=('Total', 'TOTAL'), ascending=False, inplace=True)
df_pivot_ER_CE.sort_values(by=('Total', 'TOTAL'), ascending=False, inplace=True)
#df_pivot_ER_HO.sort_values(by=('Total', 'TOTAL'), ascending=False, inplace=True)
df_pivot_HO.sort_values(by=('Total', 'TOTAL'), ascending=False, inplace=True)

# Creación del archivo .xlsx para descarga

In [ ]:
# Creación del archivo de descarga y la primera pestaña
df_pivot_CE.to_excel('MORBILIDAD_20_PRINCIPALES.xlsx', sheet_name='Consulta Externa')

In [ ]:
# Creación de las siguientes pestañas del archivo creado
with pd.ExcelWriter('MORBILIDAD_20_PRINCIPALES.xlsx', engine='openpyxl', mode='a') as writer:
    df_pivot_ER_CE.to_excel(writer, sheet_name= 'Emergencia')
#    df_pivot_ER_HO.to_excel(writer, sheet_name= 'ER HO')
    df_pivot_HO.to_excel(writer, sheet_name= 'Hospitalización')